In [5]:
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install torchsummary
!pip3 install tensorboardX

from google.colab import drive
drive.mount('/content/gdrive')
!cp gdrive/My\ Drive/x64/*.npy .
!cp gdrive/My\ Drive/*.py .
!ls -l

    100% |████████████████████████████████| 133kB 4.7MB/s 
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
total 949904
drwxr-xr-x 3 root root      4096 Mar 28 10:34 data
-rw------- 1 root root      2875 Mar 28 15:24 _dataset_tools.py
drwx------ 3 root root      4096 Mar 28 11:07 gdrive
-rw------- 1 root root      2481 Mar 28 15:24 _my_tools.py
drwxr-xr-x 2 root root      4096 Mar 28 15:19 __pycache__
drwxr-xr-x 1 root root      4096 Mar  8 17:26 sample_data
-rw------- 1 root root 129687680 Mar 28 15:23 X_test.npy
-rw------- 1 root root 518750336 Mar 28 15:24 X_train.npy
-rw------- 1 root root  64843904 Mar 28 15:24 y_test.npy
-rw------- 1 root root 259375232 Mar 28 15:24 y_train.npy


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# from tensorboardX import SummaryWriter
from torchsummary import summary

import _my_tools as mt
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device is", device)
X_train, y_train, X_test, y_test = mt.loadData("dataset_interpolation/",'float16',channels_last=False)

Device is cpu


In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.KERNEL_SIZE = 31
        self.activation = nn.ReLU()
        
        self.conv_setup = {
            'kernel' : (3,3),
            'stride' : (1,1),
            'padding' : 1,
            'activation' : self.activation
        }
        self.pooling_setup = {
            'kernel_size' : (2,2),
            'stride' : (2,2)
        }
        self.upsample_setup = {
            'scale_factor' : 2,
            'mode' : 'bilinear',
            'align_corners' : True
        }

        self.pooling_layer = nn.AvgPool2d(**self.pooling_setup)
        self.upsample_layer = nn.Upsample(**self.upsample_setup)
        
        self.conv32 = self._convBlock(2, 32, **self.conv_setup)
        self.conv64 = self._convBlock(32, 64, **self.conv_setup)
        self.conv128 = self._convBlock(64, 128, **self.conv_setup)
        self.conv256 = self._convBlock(128, 256, **self.conv_setup)
        self.conv256_256 = self._convBlock(256, 256, **self.conv_setup)


        self.upsample256 = self._upsampleBlock(self.upsample_layer, 256, 256, **self.conv_setup)
        self.deconv128 = self._convBlock(256, 128, **self.conv_setup)
        self.upsample128 = self._upsampleBlock(self.upsample_layer, 128, 128, **self.conv_setup)
        self.deconv64 = self._convBlock(128, 64, **self.conv_setup)
        self.upsample64 = self._upsampleBlock(self.upsample_layer, 64, 64, **self.conv_setup)
        self.deconv32 = self._convBlock(64, 32, **self.conv_setup)
        self.upsample32 = self._upsampleBlock(self.upsample_layer, 32, 32, **self.conv_setup)
        self.deconv1 = self._convBlock(32, 1, **self.conv_setup)
        
    def forward(self, x):
        x32 = self.conv32(x)
        x32_p = self.pooling_layer(x32)
        x64 = self.conv64(x32_p)
        x64_p = self.pooling_layer(x64)
        x128 = self.conv128(x64_p)
        x128_p = self.pooling_layer(x128)
        x256 = self.conv256(x128_p)
        x256_p = self.pooling_layer(x256)

        x = self.conv256_256(x256_p)

        # expansion

        x = self.upsample256(x)
        x += x256
        x = self.deconv128(x)

        x = self.upsample128(x)
        x += x128
        x = self.deconv64(x)

        x = self.upsample64(x)
        x += x64
        x = self.deconv32(x)
        
        x = self.upsample32(x)
        x += x32
        x = self.deconv1(x)
        
        return x
    
    @staticmethod
    def _convBlock(in_channels, out_channels, kernel, stride, padding, activation):
        return nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel, stride, padding), activation,
            nn.Conv2d(in_channels, in_channels, kernel, stride, padding), activation,
            nn.Conv2d(in_channels, out_channels, kernel, stride, padding), activation
        )
    @staticmethod
    def _upsampleBlock(upsample, in_channels, out_channels, kernel, stride, padding, activation):
        return nn.Sequential(
            upsample,
            nn.Conv2d(in_channels, out_channels, kernel, stride, padding), activation
        )

model = Model()
# print(model)
# if device == torch.device("cpu"):
#     print(summary(model, (2,64,64)))
# else:
#     print(summary(model.cuda(), (2,64,64)))
    
# --------------------------------------------------------------------------------------------------

def getLossOptimizer(net, learning_rate=0.001):
    loss_function = nn.L1Loss()
    optimizer = optim.Adamax(net.parameters(), lr=learning_rate)
    
    return (loss_function, optimizer)

def getBatch(X, y, offset, batch_size, device):
    input = torch.tensor( X[ offset:offset + batch_size ], dtype=torch.float)
    target = torch.tensor( y[ offset:offset + batch_size ], dtype=torch.float)
    return input.to(device), target.to(device)

def formatTime(t):
    s = t % 60
    m = (t // 60) % 60
    h = t // 3600
    return str(h) + ":" + str(m).zfill(2) + ":" + str(s).zfill(2)

In [4]:
import time

def fit(model, device, X, y, batch_size, n_epochs, history=None, X_val=None, y_val=None):
    
    #Print all of the hyperparameters of the training iteration:
    print("====== HYPERPARAMETERS ======")
    print("batch_size =", batch_size)
    print("epochs =", n_epochs)
    print("device",device)
    print("=" * 29)
    
    assert X.shape[0] == y.shape[0]
    assert X.shape[2:4] == y.shape[2:4]
    
    model.to(device)
    loss_function, optimizer = getLossOptimizer(model)
    n_batch = X.shape[0] // batch_size
    history = {
        'train' : [],
        'val' : []
    }
    
    start_T = int(time.time())
    tick_T = int(time.time())
    
    for epoch in range(n_epochs):
        print("===> Epoch[{}]".format(epoch), end='', flush=True)
        epoch_loss = 0
        
        for it in range(n_batch):
            input, target = getBatch(X, y, it, batch_size, device)
            
            optimizer.zero_grad()
            
            output = model(input)
            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()
            
            loss_val = loss.item()
            epoch_loss += loss_val
            
            tack_T = int(time.time())
            print("\r", end='')
            print("===> Epoch[{}]({}/{}): Loss: {:.4f}\tETA {}\tEpoch Loss: {:.4f}"
                  .format(epoch, it, n_batch, loss_val,formatTime((tack_T - tick_T) * (n_batch - it + 1)),
                  epoch_loss / (it+1)), end='', flush=True)
            tick_T = tack_T
            
        epoch_loss /= n_batch
        history['train'].append(epoch_loss)
        print("Epoch[{}] finished with loss: {}".format(epoch, epoch_loss))
        
        if X_val is not None and y_val is not None:
            history['val'].append( validate(model, loss_function, X_val, y_val) )
        
        print("\n----------------------------")
    print("Finished training of {} epochs in {}.".format(n_epochs, formatTime(int(time.time())-start_T)))
        
    return history
        
def validate(model, loss_function, X_val, y_val):
    X_val, y_val = val
    assert X_val.shape[0] == y_val.shape[0]

    total_loss_val = 0
    n_batch_val = X_val.shape[0] // batch_size

    print("Validating on {} samples.".format(n_batch_val * batch_size))

    for it in range(n_batch_val):
        input, target = getBatch(X_val, y_val, it, batch_size, device)

        output = model(input)
        loss_val = loss_function(output, target)
        total_loss_val += loss_val.data[0]

    print("Validation loss = {:.4f}".format(total_loss_val / n_batch_val))
    return total_loss_val / n_batch_val


history={}
try:
    fit(model, device, X_train, y_train, 128, 2, history, X_val=X_test, y_val=y_test)
except:
    print("\n\nFinished training.")

====== HYPERPARAMETERS ======
batch_size = 128
epochs = 2
device cpu
===> Epoch[0]

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


===> Epoch[0](3/494): Loss: 0.0902	ETA 1:05:36	Epoch Loss: 0.0904

KeyboardInterrupt: 